In [ ]:
# https://github.com/awslabs/autogluon/blob/master/examples/tabular/example_simple_tabular.py

In [9]:
from autogluon import TabularPrediction as task

## 加载数据集

In [10]:
# Training time:
train_data = task.Dataset(file_path='https://autogluon.s3-us-west-2.amazonaws.com/datasets/Inc/train.csv') # can be local CSV file as well, returns Pandas DataFrame
train_data = train_data.head(500) # subsample for faster demo

Loaded data from: https://autogluon.s3-us-west-2.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


In [11]:
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,178478,Bachelors,13,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K
1,23,State-gov,61743,5th-6th,3,Never-married,Transport-moving,Not-in-family,White,Male,0,0,35,United-States,<=50K
2,46,Private,376789,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,15,United-States,<=50K
3,55,?,200235,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,50,United-States,>50K
4,36,Private,224541,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,El-Salvador,<=50K


In [13]:
label_column = 'class' # specifies which column do we want to predict
savedir = 'data/ag_models/' # where to save trained models

predictor = task.fit(train_data=train_data, label=label_column, output_directory=savedir) # since tuning_data = None, automatically determines train/validation split
results = predictor.fit_summary() # display summary of models trained during fit()

Beginning AutoGluon training ...
Preprocessing data ...
Here are the first 10 unique label values in your data:  [' <=50K' ' >50K']
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed)
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Data preprocessing and feature engineering runtime = 0.11s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
Fitting model: RandomForestClassifierGini ...
	0.54s	 = Training runtime
	0.89	 = Validation accuracy score
Fitting model: RandomForestClassifierEntr ...
	0.42s	 = Training runtime
	0.9	 = Validation accuracy score
Fitting model: ExtraTreesClassifierGini ...
	0.42s	 = Training runtime
	0.88	 = Validation accuracy score
Fitting model: Extra

*** Summary of fit() ***
Number of models trained: 11
Types of models trained: 
{'LGBModel', 'CatboostModel', 'TabularNeuralNetModel', 'RFModel', 'WeightedEnsembleModel', 'KNNModel'}
Validation performance of individual models: {'RandomForestClassifierGini': 0.89, 'RandomForestClassifierEntr': 0.9, 'ExtraTreesClassifierGini': 0.88, 'ExtraTreesClassifierEntr': 0.86, 'KNeighborsClassifierUnif': 0.8, 'KNeighborsClassifierDist': 0.77, 'LightGBMClassifier': 0.88, 'CatboostClassifier': 0.9, 'NeuralNetClassifier': 0.86, 'LightGBMClassifierCustom': 0.89, 'weighted_ensemble_l1': 0.9}
Best model (based on validation performance): weighted_ensemble_l1
Hyperparameter-tuning used: False
Bagging used: False 
Stack-ensembling used: False 
User-specified hyperparameters:
{'NN': {'num_epochs': 500}, 'GBM': {'num_boost_round': 10000}, 'CAT': {'iterations': 10000}, 'RF': {'n_estimators': 300}, 'XT': {'n_estimators': 300}, 'KNN': {}, 'custom': ['GBM']}
Plot summary of models saved to file: SummaryOfModels

In [14]:
# Inference time:
test_data = task.Dataset(file_path='https://autogluon.s3-us-west-2.amazonaws.com/datasets/Inc/test.csv') # another Pandas DataFrame
y_test = test_data[label_column]
test_data = test_data.drop(labels=[label_column],axis=1) # delete labels from test data since we wouldn't have them in practice
print(test_data.head())

predictor = task.load(savedir) # Unnecessary, we reload predictor just to demonstrate how to load previously-trained predictor from file
y_pred = predictor.predict(test_data)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Loaded data from: https://autogluon.s3-us-west-2.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769
Evaluation: accuracy on test data: 0.824547


   age          workclass  fnlwgt      education  education-num  \
0   31            Private  169085           11th              7   
1   17   Self-emp-not-inc  226203           12th              8   
2   47            Private   54260      Assoc-voc             11   
3   21            Private  176262   Some-college             10   
4   17            Private  241185           12th              8   

        marital-status        occupation relationship    race      sex  \
0   Married-civ-spouse             Sales         Wife   White   Female   
1        Never-married             Sales    Own-child   White     Male   
2   Married-civ-spouse   Exec-managerial      Husband   White     Male   
3        Never-married   Exec-managerial    Own-child   White   Female   
4        Never-married    Prof-specialty    Own-child   White     Male   

   capital-gain  capital-loss  hours-per-week  native-country  
0             0             0              20   United-States  
1             0         

Evaluations on test data:
{
    "accuracy": 0.8245470365441704,
    "accuracy_score": 0.8245470365441704,
    "balanced_accuracy_score": 0.7085959299925461,
    "matthews_corrcoef": 0.47279226180562206,
    "f1_score": 0.8245470365441704
}
Detailed (per-class) classification report:
{
    " <=50K": {
        "precision": 0.8536555295278018,
        "recall": 0.929271238759898,
        "f1-score": 0.8898599151779977,
        "support": 7451
    },
    " >50K": {
        "precision": 0.6821471652593486,
        "recall": 0.48792062122519414,
        "f1-score": 0.5689134808853118,
        "support": 2318
    },
    "accuracy": 0.8245470365441704,
    "macro avg": {
        "precision": 0.7679013473935752,
        "recall": 0.7085959299925461,
        "f1-score": 0.7293866980316548,
        "support": 9769
    },
    "weighted avg": {
        "precision": 0.8129598197955595,
        "recall": 0.8245470365441704,
        "f1-score": 0.8137053615194404,
        "support": 9769
    }
}


In [8]:
y_pred

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' <=50K'],
      dtype=object)